<img src="https://s3-sa-east-1.amazonaws.com/preditiva.ai/diversos/preditiva_assinatura.jpg">

# Estado da Arte na Concessão de Crédito - Tuning de Hiperparâmetros

## Importação das bibliotecas

In [1]:
# Pacotes de preparação dos dados
import pandas as pd
import numpy as np
from math import sqrt
import random

# Pacotes gráficos
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Pacotes de Modelagem
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import roc_auc_score

# Funções customizadas
import preditiva

## Importação da base de dados

In [2]:
df = pd.read_csv('emprestimos.csv', sep=';')
df.head()

,Idade,Genero,Escolaridade,Tipo_Moradia,Saldo_Investimento,Saldo_Conta_Corrente,Valor_Emprestimo,Duracao_Emprestimo,Default
0,67,M,Graduacao,Propria,Sem investimento,Pouco,1169,6,0
1,22,H,Graduacao,Propria,Pouco,Moderado,5951,48,1
2,49,M,Pos Graduacao,Propria,Pouco,Sem conta,2096,12,0
3,45,M,Graduacao,De favor,Pouco,Pouco,7882,42,0
4,53,M,Graduacao,De favor,Pouco,Pouco,4870,24,1


## Preparação dos Dados

### Missing Values

In [3]:
df.isnull().sum()

Idade                   0
Genero                  0
Escolaridade            0
Tipo_Moradia            0
Saldo_Investimento      0
Saldo_Conta_Corrente    0
Valor_Emprestimo        0
Duracao_Emprestimo      0
Default                 0
dtype: int64

### Duplicações

In [4]:
df.duplicated().sum()

0

### Tipos de Variáveis

In [5]:
df.head(2)

,Idade,Genero,Escolaridade,Tipo_Moradia,Saldo_Investimento,Saldo_Conta_Corrente,Valor_Emprestimo,Duracao_Emprestimo,Default
0,67,M,Graduacao,Propria,Sem investimento,Pouco,1169,6,0
1,22,H,Graduacao,Propria,Pouco,Moderado,5951,48,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Idade                 1000 non-null   int64 
 1   Genero                1000 non-null   object
 2   Escolaridade          1000 non-null   object
 3   Tipo_Moradia          1000 non-null   object
 4   Saldo_Investimento    1000 non-null   object
 5   Saldo_Conta_Corrente  1000 non-null   object
 6   Valor_Emprestimo      1000 non-null   int64 
 7   Duracao_Emprestimo    1000 non-null   int64 
 8   Default               1000 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


## Análise Exploratória dos Dados

### Dispersão por Classe

In [ ]:
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')

sns.pairplot(df, hue='Default');

### Relatórios Pandas Profiling e SweetViz

In [ ]:
perfil_pp, perfil_sv = preditiva.gera_relatorios_aed(df=df,
                                                     target_feat='Default')
sns.reset_defaults()

## Desenvolvimento dos Modelos

### Definição do *Target* e das *Features*

In [ ]:
df.info()

In [7]:
# Target (variável resposta)
y_var = 'Default'
y = df[y_var]

# Features (variáveis explicativas)
# Variáveis Numéricas
x_var_num = [
    'Idade', 'Valor_Emprestimo', 'Duracao_Emprestimo'
    ]
x_num = df[x_var_num].to_numpy()

# Variáveis Categóricas / Qualitativas
x_var_cat = [
    'Genero', 'Escolaridade', 'Tipo_Moradia',
    'Saldo_Investimento','Saldo_Conta_Corrente'
            ]
x_cat = df[x_var_cat]

### Pré-processamento

In [8]:
# Criação das variáveis dummies para variáveis categóricas
x_cat_enc = OneHotEncoder()
x_cat = x_cat_enc.fit_transform(x_cat).toarray()

# Unificação da base de dados
x = np.concatenate([x_num, x_cat], axis=1)

### Divisão das bases em Treino e Teste

In [9]:
# Divisão em treino e teste
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.30,
                                                    random_state=42)

### Desenvolvimento e Treinamento do Gradient Boosting Baseline

In [10]:
# Construção do modelo Tensorflow encapsulado como Scikit-Learn
def modelo_GB(min_samples_leaf=10, max_depth=3, n_estimators=50, subsample=0.5):
    
    # Definição da estrutura do modelo
    model = GradientBoostingClassifier(
        n_estimators=n_estimators, 
        subsample=subsample,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth, 
        random_state=42
    )
    
    return model

In [11]:
modelo_bl = modelo_GB()
modelo_bl.fit(x_train, y_train)

GradientBoostingClassifier(min_samples_leaf=10, n_estimators=50,
                           random_state=42, subsample=0.5)

### Análise de Desempenho

In [12]:
modelo_bl_desemp = preditiva.calcula_desempenho(modelo_bl,
                                                x_train, y_train,
                                                x_test, y_test)
modelo_bl_desemp

,Treino,Teste,Variação
Acurácia,0.835714,0.750000,-0.10
AUROC,0.876465,0.761291,-0.13
KS,0.597121,0.440139,-0.26
Precision,0.845588,0.648148,-0.23
Recall,0.550239,0.384615,-0.30
F1,0.666667,0.482759,-0.28


## Definição do espaço hiperparamétrico

In [13]:
5*4*6*5

600

In [14]:
# Número mínimo de amostras por folha
ehp_min_samples_leaf = np.arange(5, 26, 5)

# Máxima profundidade
ehp_max_depth = [2, 3, 4, 5]

# Número de estimadores
ehp_n_estimators = [10, 20, 50, 100, 150, 200]

# Amostragem de observações usadas para cada árvore
ehp_subsample = [0.1, 0.2, 0.3, 0.4, 0.5]

# Definição do dicionário de parâmetros
param_grid = dict(min_samples_leaf=ehp_min_samples_leaf,
                  max_depth=ehp_max_depth,
                  n_estimators=ehp_n_estimators,
                  subsample=ehp_subsample
                 )

In [15]:
param_grid

{'min_samples_leaf': array([ 5, 10, 15, 20, 25]),
 'max_depth': [2, 3, 4, 5],
 'n_estimators': [10, 20, 50, 100, 150, 200],
 'subsample': [0.1, 0.2, 0.3, 0.4, 0.5]}

## Random Search

In [16]:
# Treinamento dos modelos usando Random Search
modelo_base = modelo_GB()
modelo_random = RandomizedSearchCV(estimator=modelo_base, 
                                   param_distributions=param_grid,
                                   n_iter=50,
                                   n_jobs=5,
                                   cv=2,
                                   verbose=4,
                                   random_state=42)

modelo_random.fit(x_train, y_train)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


RandomizedSearchCV(cv=2,
                   estimator=GradientBoostingClassifier(min_samples_leaf=10,
                                                        n_estimators=50,
                                                        random_state=42,
                                                        subsample=0.5),
                   n_iter=50, n_jobs=5,
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'min_samples_leaf': array([ 5, 10, 15, 20, 25]),
                                        'n_estimators': [10, 20, 50, 100, 150,
                                                         200],
                                        'subsample': [0.1, 0.2, 0.3, 0.4, 0.5]},
                   random_state=42, verbose=4)

In [17]:
modelo_random.best_estimator_

GradientBoostingClassifier(max_depth=2, min_samples_leaf=15, n_estimators=200,
                           random_state=42, subsample=0.2)

In [18]:
modelo_random_desemp = preditiva.calcula_desempenho(modelo_random.best_estimator_,
                                                    x_train, y_train,
                                                    x_test,  y_test)
modelo_random_desemp

,Treino,Teste,Variação
Acurácia,0.787143,0.726667,-0.08
AUROC,0.842758,0.753878,-0.11
KS,0.578986,0.445292,-0.23
Precision,0.711268,0.578947,-0.19
Recall,0.483254,0.362637,-0.25
F1,0.575499,0.445946,-0.23


## Grid Search

In [19]:
# Treinamento dos modelos usando Random Search
modelo_grid = GridSearchCV(estimator=modelo_base, 
                           param_grid=param_grid,
                           n_jobs=4,
                           cv=2,
                           verbose=4)

modelo_grid.fit(x_train, y_train)

Fitting 2 folds for each of 600 candidates, totalling 1200 fits


GridSearchCV(cv=2,
             estimator=GradientBoostingClassifier(min_samples_leaf=10,
                                                  n_estimators=50,
                                                  random_state=42,
                                                  subsample=0.5),
             n_jobs=4,
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_leaf': array([ 5, 10, 15, 20, 25]),
                         'n_estimators': [10, 20, 50, 100, 150, 200],
                         'subsample': [0.1, 0.2, 0.3, 0.4, 0.5]},
             verbose=4)

In [20]:
modelo_grid.best_estimator_

GradientBoostingClassifier(max_depth=2, min_samples_leaf=15, n_estimators=150,
                           random_state=42, subsample=0.2)

In [21]:
modelo_grid_desemp = preditiva.calcula_desempenho(modelo_grid.best_estimator_,
                                                  x_train, y_train, 
                                                  x_test,  y_test)
modelo_grid_desemp

,Treino,Teste,Variação
Acurácia,0.795714,0.740000,-0.07
AUROC,0.832331,0.774226,-0.07
KS,0.556486,0.458068,-0.18
Precision,0.722973,0.603175,-0.17
Recall,0.511962,0.417582,-0.18
F1,0.599440,0.493506,-0.18


## Bayesian Optimization

In [30]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import sys

from sklearn.metrics import recall_score

In [31]:
param_grid

{'min_samples_leaf': array([ 5, 10, 15, 20, 25]),
 'max_depth': [2, 3, 4, 5],
 'n_estimators': [10, 20, 50, 100, 150, 200],
 'subsample': [0.1, 0.2, 0.3, 0.4, 0.5]}

In [33]:
bo_space = {'min_samples_leaf': hp.choice('min_samples_leaf', param_grid['min_samples_leaf']),
            'max_depth': hp.choice('max_depth', param_grid['max_depth']),
            'n_estimators': hp.choice('n_estimators', param_grid['n_estimators']),
            'subsample': hp.choice('subsample', param_grid['subsample'])
           }

In [34]:
def bo_gb(param):   

    print ('Hiperparametros testados: ', param)
    modelo_bo = modelo_GB(min_samples_leaf=param['min_samples_leaf'],
                          max_depth=param['max_depth'],
                          n_estimators=param['n_estimators'],
                          subsample=param['subsample']
                         )
    
    modelo_bo.fit(x_train, y_train)
    
    # Calcula AuROC no conjunto de Teste
    y_test_pred_proba = modelo_bo.predict_proba(x_test)[:,1]
    y_test_pred = modelo_bo.predict(x_test)
    recall_test = recall_score(y_test, y_test_pred)
    auroc_test = roc_auc_score(y_test, y_test_pred_proba)
    
    print(f'AuROC Teste: {auroc_test} | Recall Teste: {recall_test}')
    
    # sys.stdout.flush() 
    return {'loss': (-recall_test), 'status': STATUS_OK}

In [35]:
trials = Trials()
bo_search = fmin(fn=bo_gb, 
                 space=bo_space,
                 algo=tpe.suggest,
                 max_evals=100,
                 trials=trials)

Hiperparametros testados:                                                                                                                                                                                                                                                           
{'max_depth': 3, 'min_samples_leaf': 25, 'n_estimators': 20, 'subsample': 0.2}                                                                                                                                                                                                      
AuROC Teste: 0.7606078132393922 | Recall Teste: 0.12087912087912088                                                                                                                                                                                                                 
Hiperparametros testados:                                                                                                                                                

In [36]:
from hyperopt import space_eval

bo_melhor_hp = space_eval(bo_space, bo_search)
print('Melhor conjunto de hiperparâmetros: ', bo_melhor_hp)

Melhor conjunto de hiperparâmetros:  {'max_depth': 5, 'min_samples_leaf': 10, 'n_estimators': 150, 'subsample': 0.1}


In [37]:
modelo_bo = modelo_GB(min_samples_leaf=bo_melhor_hp['min_samples_leaf'],
                          max_depth=bo_melhor_hp['max_depth'],
                          n_estimators=bo_melhor_hp['n_estimators'],
                          subsample=bo_melhor_hp['subsample']
                     )

historico = modelo_bo.fit(x_train, y_train)

In [38]:
modelo_bo_desemp = preditiva.calcula_desempenho(modelo_bo,
                                                x_train, y_train,
                                                x_test,  y_test)
modelo_bo_desemp

,Treino,Teste,Variação
Acurácia,0.811429,0.736667,-0.09
AUROC,0.841438,0.768547,-0.09
KS,0.555384,0.464430,-0.16
Precision,0.697436,0.575000,-0.18
Recall,0.650718,0.505495,-0.22
F1,0.673267,0.538012,-0.20
